In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sin, cos, sqrt, atan2, radians

import os
import time

print(os.listdir("../input"))

In [ ]:
main_file = pd.read_excel('../input/MT.xlsx')
main_file.head()
# main_file = main_file[0:500]

In [ ]:
R = 6373.0
def distance(x,y):
    try:
        x = x.split(',')
        y = y.split(',')
        x[0] = float(x[0])
        x[1] = float(x[1])
        y[0] = float(y[0])
        y[1] = float(y[len(y)-1])
    except ValueError:
        print(x,y)
        return 100000
    except AttributeError:
        print(x,y)
        return 100000
    x = [x[0],x[1]]
    y = [y[0],y[1]]
    #---------------
    lat_a, long_a, lat_b, long_b = map(radians, [*x,*y])    #[*x,*y]
    dlon = long_b - long_a
    dlat = lat_b - lat_a
    a = sin(dlat/2)**2 + cos(lat_a) * cos(lat_b) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return (R * c)

In [ ]:
# x='46.940995,142.738319,'
# x=x.split(',')
# x[0] = float(x[0])
# x[1] = float(x[1])
# x=[x[0],x[1]]
# x

In [ ]:
scep_koord = main_file['Сцеп коорд']
scep_koord = scep_koord.dropna()
scep_koord_v = scep_koord.values

In [ ]:
cols = main_file.columns[1:31]

for col in cols:
    f_col = main_file[col]
    f_col = f_col.dropna()
#     f_col.replace("'', '",',')
    f_col_v = f_col.values
    
    start_time = time.time()
    
    c50 = []
    c100 = []
    c250 = []
    c500 = []
    
    for elem in scep_koord_v:
        c_50 = 0
        c_100 = 0
        c_250 = 0
        c_500 = 0
        for el in f_col_v:
            dist = distance(elem,el)
            if dist<=0.05:
                c_50  = c_50  + 1
            if dist<=0.1:
                c_100  = c_100  + 1
            if dist<=0.25:
                c_250  = c_250  + 1
            if dist<=0.5:
                c_500  = c_500  + 1
        c50.append(c_50)
        c100.append(c_100)
        c250.append(c_250)
        c500.append(c_500)
        
    d = {str(col)+'_50м': c50, str(col)+'_100м': c100, str(col)+'_250м': c250, str(col)+'_500м': c500}
    new_df = pd.DataFrame(data=d)
    
    scep_koord = pd.concat([scep_koord, new_df], axis=1)

    print("--- %s seconds --- " % (time.time() - start_time), col)

In [ ]:
scep_koord

In [ ]:
# submit = pd.DataFrame(gbm.predict(X_test), index=test_ind.index,columns=['target'])
scep_koord.to_csv('distances_MT.csv')